In [ ]:
import os
from typing import Annotated, Literal

from pydantic import BaseModel

from autogen.agentchat import ConversableAgent, UserProxyAgent
from autogen.tools.dependency_injection import BaseContext, ChatContext, Depends

In [2]:
class Account(BaseContext, BaseModel):
    username: str
    password: str
    currency: Literal["USD", "EUR"] = "USD"


alice_account = Account(username="alice", password="password123")
bob_account = Account(username="bob", password="password456")

account_ballace_dict = {
    (alice_account.username, alice_account.password): 300,
    (bob_account.username, bob_account.password): 200,
}

In [3]:
def _verify_account(account: Account):
    if (account.username, account.password) not in account_ballace_dict:
        raise ValueError("Invalid username or password")


def _get_balance(account: Account):
    _verify_account(account)
    return f"Your balance is {account_ballace_dict[(account.username, account.password)]}{account.currency}"

In [4]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
agent = ConversableAgent(
    name="agent",
    llm_config={"config_list": config_list},
)
user_proxy = UserProxyAgent(
    name="user_proxy_1",
    human_input_mode="NEVER",
    llm_config=False,
)

In [5]:
from logging import Logger


@user_proxy.register_for_execution()
@agent.register_for_llm(description="Login")
def login(
    account: Annotated[Account, Depends(bob_account)],
    logger: Logger,
) -> str:
    _verify_account(account)
    logger.info(f"User {account.username} logged in")
    return "You are logged in"




@user_proxy.register_for_execution()
@agent.register_for_llm(description="Get balance")
def get_balance_1(
    account: Annotated[Account, Depends(bob_account)],
    chat_context: ChatContext,
    logger: Logger,
) -> str:
    logger.info("Hello, World!")

    messages_with_first_agent = list(chat_context.chat_messages.values())[0]

    login_function_called = False
    for message in messages_with_first_agent:
        if "tool_calls" in message and message["tool_calls"][0]["function"]["name"] == "login":
            login_function_called = True
            break

    if not login_function_called:
        logger.error("Login function was not called")
        raise ValueError("Login function was not called")

    balance = _get_balance(account)
    logger.info("Successfully retrieved balance")
    return balance


In [ ]:
user_proxy.initiate_chat(agent, message="Get users balance", max_turns=5)

In [7]:
messages_with_first_agent = list(user_proxy.chat_messages.values())[0]

login_function_called = False
for message in messages_with_first_agent:
    if "tool_calls" in message and message["tool_calls"][0]["function"]["name"] == "login":
        login_function_called = True
        break

if not login_function_called:
    raise ValueError("Login function was not called")
